# Verification: LBFGS in vcl.StandardJet framework vs general LBFGS framework

## LBFGS in vcl.StandardJet framework 

In [1]:
import vcl
import npvc
import vcalg
import optimizationAlg_2

sp = npvc.Space(4)

x = vcl.Vector(sp)
x.data[0]=-1.2
x.data[1]=1.0
x.data[2]=-1.2
x.data[3]=1.0

b = vcl.Vector(sp)
b.data[0]=0
b.data[1]=-1
b.data[2]=0
b.data[3]=-1

F = npvc.DoubleRosie(sp)
lsq = vcl.LeastSquares(F,b)

print('\n****************** ')
print('\n LBFGS test')
print('\n****************** ')


m = 5
alpha = 1.0
c = 0.0001
iterMax = 100
verbose = 1
beta = 0.5
eps = 0.00000000000001
Kmax = 1200
tol = 0.00001
plot_on = 0 #  plot for the solution
gamma = 1
x = optimizationAlg_2.lbfgs(vcl.StandardJet, x, m, sp, sp, iterMax, 
                        alpha, beta, c, verbose, eps, gamma, tol,
                        Kmax, plot_on, jetargs=dict(fcn=lsq))

print('\nSolution')
x.myNameIs()



****************** 

 LBFGS test

****************** 
 
Iteration 1 of LBFGS:
i is the iteration counter for line search.
 
Before Line Search:
i = 0 alpha = 0 f(x) =  14.9072 dfp = -13601.205376
 
During Line Search:
 
fxvalue is 14.9072
i=0    alpha=1.0   f(x) =  14.9072   f(x)-f(xn)=-6405496796.832956
||p||=116.62420578936432     ||df||=116.62420578936432    dfp=-13601.205376 
c*alpha*dfp=-1.3601205376
 
 
fxvalue is 14.9072
i=1    alpha=0.5   f(x) =  14.9072   f(x)-f(xn)=-379307001.6565687
||p||=116.62420578936432     ||df||=116.62420578936432    dfp=-13601.205376 
c*alpha*dfp=-0.6800602688
 
 
fxvalue is 14.9072
i=2    alpha=0.25   f(x) =  14.9072   f(x)-f(xn)=-21194413.74786695
||p||=116.62420578936432     ||df||=116.62420578936432    dfp=-13601.205376 
c*alpha*dfp=-0.3400301344
 
 
fxvalue is 14.9072
i=3    alpha=0.125   f(x) =  14.9072   f(x)-f(xn)=-1039382.4309733049
||p||=116.62420578936432     ||df||=116.62420578936432    dfp=-13601.205376 
c*alpha*dfp=-0.1700150672
 
 
fxv

## General LBFGS framework

In [2]:
import vcl
import npvc
import vcalg
import linalg
import numpy as np
import func
import lsalg

# total gradient evaluation
gtot = 0
#total backtracking steps
ktot = 0
#total function evaluation
jtot = 0

dom = npvc.Space(4)
rng = npvc.Space(4)



s_set = []
y_set = []
s = vcl.Vector(dom)
y = vcl.Vector(dom)
df = vcl.Vector(dom)
df_old = vcl.Vector(dom)
q = vcl.Vector(dom)
p = vcl.Vector(dom)
x_old = vcl.Vector(dom)
r = vcl.Vector(dom)

# the number of vector pair {s,y} we want to keep
m = 5
k = 0

f = func.DoubleRosie(dom,rng)
x = vcl.Vector(dom)
b = vcl.Vector(rng)
b.data[0]=0
b.data[1]=-1
b.data[2]=0
b.data[3]=-1
x.data[0]=-1.2
x.data[1]=1.0
x.data[2]=-1.2
x.data[3]=1.0
J = vcl.LeastSquares(f,b)
df=J.gradient(x)
df_old.copy(df)


m = 5
alpha = 1.0
c = 0.0001
iterMax = 100
verbose = 1
beta = 0.5
eps = 0.00000000000001
Kmax = 1200
tol = 0.00001
plot_on = 0 #  plot for the solution
gamma = 1


while df.norm() > tol and k <= Kmax:
    p = lsalg.descent_direction_1(df, k, m, s_set, y_set, gamma)
    x_old.copy(x)
    [i, x, jtot, ktot] = lsalg.bt_lineSearch_1(J, x, df, p, k, iterMax, alpha, beta, c, verbose, jtot, ktot)
    
    if plot_on > 0:
        print(" ")
        print("The updated slowness: ")
        print(" ")
        linalg.simplot(mest.data, addcb=True, minval=2.5, maxval=4.5)
        
    if i == iterMax:
        print("The line search failed.")
        break
    f_obj = J(x)
    df_old.copy(df)
    df = J.gradient(x)
    gtot += 1
    # s_k = x_{k+1} - x_k
    s.copy(x)
    s.linComb(-1,x_old,1)
    # y_k = df_{k+1} - df_k
    y.copy(df)
    y.linComb(-1,df_old,1)

    if k < m:
        s_set.append(s.dup())
        y_set.append(y.dup())

    
    if k >= m and s.dot(y) > eps*s.norm()*y.norm():
        s_set.pop(0)
        s_set.append(s.dup())
        y_set.pop(0)
        y_set.append(y.dup())     
    gamma = s_set[-1].dot(y_set[-1])/y_set[-1].dot(y_set[-1])
    k += 1
    if k >= Kmax:
        print("The BFGFS algorithm reached the maximum number of iterations.")
        break


print(f"total function evals = {jtot}")
print(f"total gradient evals = {gtot}")
print(f"total backtracking steps = {ktot}")
x.myNameIs()

 
Iteration 1 of LBFGS:
i is the iteration counter for line search.
 
Before Line Search:
i = 0 alpha = 0 f(x) =  + 14.9072 dfp = -13601.205376
 
During Line Search:
 
i=0    alpha=1.0   f(x)=14.9072    f(x)-f(xn)=-6405496796.832956
||p||=116.62420578936432     ||df||=116.62420578936432    dfp=-13601.205376 
c*alpha*dfp=-1.3601205376
 
 
i=1    alpha=0.5   f(x)=14.9072    f(x)-f(xn)=-379307001.6565687
||p||=116.62420578936432     ||df||=116.62420578936432    dfp=-13601.205376 
c*alpha*dfp=-0.6800602688
 
 
i=2    alpha=0.25   f(x)=14.9072    f(x)-f(xn)=-21194413.74786695
||p||=116.62420578936432     ||df||=116.62420578936432    dfp=-13601.205376 
c*alpha*dfp=-0.3400301344
 
 
i=3    alpha=0.125   f(x)=14.9072    f(x)-f(xn)=-1039382.4309733049
||p||=116.62420578936432     ||df||=116.62420578936432    dfp=-13601.205376 
c*alpha*dfp=-0.1700150672
 
 
i=4    alpha=0.0625   f(x)=14.9072    f(x)-f(xn)=-36215.36835520786
||p||=116.62420578936432     ||df||=116.62420578936432    dfp=-13601.205

 
Iteration 28 of LBFGS:
i is the iteration counter for line search.
 
Before Line Search:
i = 0 alpha = 0 f(x) =  + 0.06654569032571095 dfp = -0.01703703419758519
 
During Line Search:
 
i=0    alpha=1.0   f(x)=0.06654569032571095    f(x)-f(xn)=0.014157929529207053
||p||=0.10430154981531334     ||df||=0.43562553960245315    dfp=-0.01703703419758519 
c*alpha*dfp=-1.7037034197585192e-06
 
k = 28 alpha = 1.0 f(xn) = 0.052387760796503896 ||df|| = 0.8053036589589706
The total run time for this line search is: 0.06205582618713379 sec.
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
 
Iteration 29 of LBFGS:
i is the iteration counter for line search.
 
Before Line Search:
i = 0 alpha = 0 f(x) =  + 0.052387760796503896 dfp = -0.028735788842181295
 
During Line Search:
 
i=0    alpha=1.0   f(x)=0.052387760796503896    f(x)-f(xn)=0.010140399548894494
||p||=0.2510997224670241     ||df||=0.